In [187]:
# IMPORT STATEMENTS
import pandas as pd
import matplotlib as plt

In [255]:
# SAVE KAPRSAC JET FUEL PRICES IN A DATAFRAME
df_org = pd.read_csv("KARPSAC-jetfuelprices-2010to2020.csv", sep=";") # original data set
df = df_org.copy() 

In [99]:
# PRELIMINARY STATS:
df.describe()

,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB in U.S. Dollars per Gallon
count,7626.000000
mean,1.407106
std,0.897850
min,0.282000
25%,0.581000
50%,1.199000
75%,2.007750
max,4.814000


In [256]:
# CLEAN UP DATA TO PERFORM GROUPBY ANALYSIS
df['YEAR'] = df['Date'].astype(str).str[0:4]
df = df[df['YEAR'].astype(int) >= 2010] # filters only years from 2010 to 2020
df['MONTH'] = df['Date'].astype(str).str[5:7]
df['DAY'] = df['Date'].astype(str).str[8:10]
del df['Date']

In [257]:
#CHANGE NUMERICAL MONTH TO MONTHS
month_lookup = {'01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May',
                '06': 'June', '07': 'July', '08': 'August', '09': 'September', '10': 'October',
                '11': 'November','12': 'December'}

In [258]:
df['MONTH'] = df['MONTH'].apply(lambda x: month_lookup[x])

In [259]:
df['MONTH YEAR'] = df['MONTH'] + ' ' +  df['YEAR']
df.set_index('MONTH YEAR')

,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB in U.S. Dollars per Gallon,YEAR,MONTH,DAY
MONTH YEAR,,,,
August 2012,3.123,2012,August,07
August 2012,3.051,2012,August,05
August 2012,2.939,2012,August,01
July 2012,2.914,2012,July,30
July 2012,2.934,2012,July,15
...,...,...,...,...
September 2012,3.218,2012,September,05
August 2012,3.225,2012,August,26
August 2012,3.162,2012,August,19


In [260]:
# FIND MEAN OF THESE PRICES
jet_fuel = df.groupby(['MONTH YEAR']).mean()

In [310]:
# IMPORT BTS DOMESTICS DATA DEPARTURES PERFORMED
dom_flight = pd.read_csv('USCarrier_Traffic_DOM.csv', sep='","', engine='python', skiprows=1)
dom_flight.drop(dom_flight.tail(1).index,inplace=True) # drop last column

In [311]:
# RENAME COLUMNS
dom_flight.index.name = 'Period'
dom_flight.rename(columns={'Unit: (000)': 'Total(DOM)'}, inplace=True)

In [312]:
# CLEANUP DATA
dom_flight['PERIOD'] = dom_flight.index
dom_flight['PERIOD'] = dom_flight['PERIOD'].astype(str).str[1:]
dom_flight.set_index('PERIOD')
dom_flight['Total(DOM)'] = dom_flight['Total(DOM)'].astype(str).str[:-1]

In [313]:
dom_flight.set_index('PERIOD', inplace=True)

In [315]:
dom_flight.head()

,Total(DOM)
PERIOD,
July 2010,798813
August 2010,799094
September 2010,733875
October 2010,749822
November 2010,714897


In [316]:
# IMPORT BTS INTERNATIONAL DATA DEPARTURES PERFORMED
intl_flight = pd.read_csv('USCarrier_Traffic_INTL.csv', sep='","', engine='python', skiprows=1)
intl_flight.drop(intl_flight.tail(1).index,inplace=True)

In [317]:
# RENAME COLUMNS
intl_flight.index.name = 'Period'
intl_flight.rename(columns={'Unit: (000)': 'Total(INTL)'}, inplace=True)

In [318]:
# CLEANUP DATA
intl_flight['PERIOD'] = intl_flight.index
intl_flight['PERIOD'] = intl_flight['PERIOD'].astype(str).str[1:]
intl_flight.set_index('PERIOD')
intl_flight['Total(INTL)'] = intl_flight['Total(INTL)'].astype(str).str[:-1]

In [319]:
intl_flight.set_index('PERIOD', inplace=True)

In [320]:
intl_flight.head()

,Total(INTL)
PERIOD,
July 2010,81903
August 2010,78229
September 2010,64405
October 2010,66413
November 2010,64579


In [335]:
# CREATE COMBINED FLIGHT DATAFRAME
combined_flight = dom_flight.copy()

In [336]:
combined_flight['Total(INTL)'] = intl_flight['Total(INTL)']

In [339]:
combined_flight['TOTAL'] = combined_flight['Total(DOM)'].astype(int) + combined_flight['Total(INTL)'].asty

In [340]:
combined_flight

,Total(DOM),Total(INTL),TOTAL
PERIOD,,,
July 2010,798813,81903,79881381903
August 2010,799094,78229,79909478229
September 2010,733875,64405,73387564405
October 2010,749822,66413,74982266413
November 2010,714897,64579,71489764579
...,...,...,...
December 2019,729109,74909,72910974909
January 2020,708984,73377,70898473377
February 2020,668537,67801,66853767801
